<a href="https://colab.research.google.com/github/gomescheelsy03/INST-414-0101-/blob/main/W5_in_class_exercise_CGomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Name: Chelsy Gomes
# Instructor: Cody Buntain
# Course: INST414 (0101)
# Date: 10/10/2025
# Assignment: In-Class Exercise Week 5

Exercise 1. Normalizing Actor-Genre Matrix

In [2]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.metrics import DistanceMetric, pairwise_distances

actor_name_map = {}
movie_actor_map = {}
actor_genre_map = {}

with open("imdb_movies_2000to2022.prolific.json", "r", encoding="utf-8") as in_file:
  for line in in_file:
    this_movie= json.loads(line)

    for actor_id, actor_name in this_movie['actors']:
      actor_name_map[actor_id] = actor_name

    for actor_id, actor_name in this_movie['actors']:
      this_actors_genres = actor_genre_map.get(actor_id, {})
      for g in this_movie["genres"]:
        this_actors_genres[g] = this_actors_genres.get(g, 0)+1
      actor_genre_map[actor_id] = this_actors_genres

    movie_actor_map[this_movie["imdb_id"] ] = {
        "movie": this_movie["title"],
        "actors": set([item[0] for item in this_movie['actors']]),
        "genres": this_movie["genres"]
    }
index = actor_name_map.keys()
rows= [actor_genre_map[k] for k in index]
df= pd.DataFrame(rows, index=index).fillna(0).astype(float)

row_sums= df.sum(axis=1)
safe_mask=row_sums>0
df=df.loc[safe_mask]
row_sums=row_sums.loc[safe_mask]
df_norm=df.divide(row_sums,axis=0)

target_actor_id= 'nm1165110'

euclid= DistanceMetric.get_metric("euclidean")
all_vecs= df_norm.to_numpy()
target_vec=df_norm.loc[target_actor_id].to_numpy().reshape(1,-1)
distances= euclid.pairwise(all_vecs, target_vec) [:,0]

pairs= [(aid,d) for aid,d in zip(df_norm.index.tolist(),distances) if aid != target_actor_id]
top10= sorted(pairs, key=lambda x: x[1]) [:10]

print("euclidean distance on L1 Normalization")
for aid, d in top10:
  print(aid, actor_name_map.get(aid, "Unknown"), d, int(df.loc[aid].sum()))

euclidean distance on L1 Normalization
nm0000129 Tom Cruise 0.09789754549980823 61
nm0829032 Ray Stevenson 0.12496192961003849 23
nm0147147 Henry Cavill 0.12797458635598966 37
nm5899377 Tiger Shroff 0.134492704809032 20
nm0003244 Jordi Mollà 0.13472631787668815 15
nm1679372 Sudeep 0.1359654753899035 15
nm2018237 Taylor Kitsch 0.14047239052070495 21
nm4043618 Tom Holland 0.14238483556047632 34
nm2207222 Scott Eastwood 0.14953462799296724 29
nm5744729 Vaani Kapoor 0.15437392795883362 11


In [3]:
cosine_distances= pairwise_distances(df,df.loc[[target_actor_id]], metric="cosine") [:,0]
cosine_pairs= [(aid,d) for aid,d in zip(df.index, cosine_distances) if aid != target_actor_id]
cosine_top10=sorted(cosine_pairs, key= lambda x: x[1]) [:10]

print("Top 10 most similar actors using Cosine similarity")
for aid,d in cosine_top10:
      print(aid, actor_name_map.get(aid, "Unknown"), f"dists={d:.4f}", int(df.loc[aid].sum()))

Top 10 most similar actors using Cosine similarity
nm0000129 Tom Cruise dists=0.0269 61
nm0147147 Henry Cavill dists=0.0446 37
nm0829032 Ray Stevenson dists=0.0452 23
nm5899377 Tiger Shroff dists=0.0488 20
nm1679372 Sudeep dists=0.0517 15
nm0003244 Jordi Mollà dists=0.0534 15
nm0636280 Richard Norton dists=0.0565 4
nm0607884 Mark Mortimer dists=0.0565 4
nm2018237 Taylor Kitsch dists=0.0580 21
nm4043618 Tom Holland dists=0.0602 34


Extra Practice. Adding Movie-Vote Counts to Actor-Genre Matrix

In [8]:
# ---------- 1) Load data and build maps ----------
actor_name_map = {}
actor_genre_map = {}
actor_votes_map = {}

dataset_path = "imdb_movies_2000to2022.prolific.json"

with open(dataset_path, "r", encoding="utf-8") as f:
    for line in f:
        m = json.loads(line)

        votes = 0
        r = m.get("rating")
        if isinstance(r, dict):
            votes = int(r.get("votes", 0) or 0)

        genres = m.get("genres", []) or []
        for actor_id, actor_name in m.get("actors", []) or []:
            actor_name_map[actor_id] = actor_name

            gcounts = actor_genre_map.get(actor_id, {})
            for g in genres:
                gcounts[g] = gcounts.get(g, 0) + 1
            actor_genre_map[actor_id] = gcounts

            actor_votes_map[actor_id] = actor_votes_map.get(actor_id, 0) + votes


# ---------- 2) Actor x Genre dataframe ----------
index = list(actor_name_map.keys())
rows = [actor_genre_map.get(k, {}) for k in index]
df = pd.DataFrame(rows, index=index).fillna(0).astype(float)

# ---------- 3) L1 row normalization ----------
row_sums = df.sum(axis=1)
safe = row_sums > 0
df.loc[safe] = df.loc[safe].div(row_sums[safe], axis=0)

# ---------- 4) Pick query actor ----------
target_actor_id = "nm1165110"  # Chris Hemsworth

# ---------- 5) Euclidean on normalized genres (baseline) ----------
euclid = DistanceMetric.get_metric("euclidean")
all_vecs = df.to_numpy()
qv = df.loc[target_actor_id].to_numpy().reshape(1, -1)
dists = euclid.pairwise(all_vecs, qv)[:, 0]
pairs = [(aid, d) for aid, d in zip(df.index, dists) if aid != target_actor_id]
top10 = sorted(pairs, key=lambda x: x[1])[:10]

print("Euclidean distance on L1-normalized Genre Matrix (no votes)")
for aid, d in top10:
    print(aid, actor_name_map.get(aid, "Unknown"), f"{d:.6f}")

# ---------- 6) Add the 'votes' column (total votes per actor) ----------
df_with_votes = df.copy()
df_with_votes["votes"] = df_with_votes.index.map(lambda aid: actor_votes_map.get(aid, 0))


all_vecs_v = df_with_votes.to_numpy()
qv_v = df_with_votes.loc[target_actor_id].to_numpy().reshape(1, -1)
dists_v = euclid.pairwise(all_vecs_v, qv_v)[:, 0]
pairs_v = [(aid, d) for aid, d in zip(df_with_votes.index, dists_v) if aid != target_actor_id]
top10_v = sorted(pairs_v, key=lambda x: x[1])[:10]

print("\nEuclidean distance AFTER adding raw 'votes' column (unscaled)")
for aid, d in top10_v:
    print(aid, actor_name_map.get(aid, "Unknown"), f"{d:.6f}", "votes", actor_votes_map.get(aid, 0))

# ---------- 7) Min/Max normalize only the 'votes' column, then recalc ----------
df_votes_scaled = df_with_votes.copy()
vmin, vmax = df_votes_scaled["votes"].min(), df_votes_scaled["votes"].max()
if vmax > vmin:
    df_votes_scaled["votes"] = (df_votes_scaled["votes"] - vmin) / (vmax - vmin)
else:
    df_votes_scaled["votes"] = 0.0


all_vecs_s = df_votes_scaled.to_numpy()
qv_s = df_votes_scaled.loc[target_actor_id].to_numpy().reshape(1, -1)
dists_s = euclid.pairwise(all_vecs_s, qv_s)[:, 0]
pairs_s = [(aid, d) for aid, d in zip(df_votes_scaled.index, dists_s) if aid != target_actor_id]
top10_s = sorted(pairs_s, key=lambda x: x[1])[:10]

print("\nEuclidean distance AFTER adding 'votes' with Min/Max normalization")
for aid, d in top10_s:
    print(aid, actor_name_map.get(aid, "Unknown"), f"{d:.6f}", "norm_votes", f"{df_votes_scaled.loc[aid, 'votes']:.4f}")

# ---------- 8) One-paragraph comparison (printed) ----------
print("\nComparison:")
print(
    "Without votes, similarity comes only from genre distribution.\n"
    "Adding raw vote totals changes rankings a lot because the votes column has a much larger scale than the genre features,\n"
    "so distance is dominated by popularity.\n"
    "After applying Min/Max scaling to the votes column (0–1 range), the rankings become more balanced,\n"
    "reflecting both genre patterns and relative popularity without letting votes overwhelm the metric."
)


Euclidean distance on L1-normalized Genre Matrix (no votes)
nm0000129 Tom Cruise 0.097898
nm0829032 Ray Stevenson 0.124962
nm0147147 Henry Cavill 0.127975
nm5899377 Tiger Shroff 0.134493
nm0003244 Jordi Mollà 0.134726
nm1679372 Sudeep 0.135965
nm2018237 Taylor Kitsch 0.140472
nm4043618 Tom Holland 0.142385
nm2207222 Scott Eastwood 0.149535
nm5744729 Vaani Kapoor 0.154374

Euclidean distance AFTER adding raw 'votes' column (unscaled)
nm0000323 Michael Caine 16188.000003 votes 7851747
nm0000242 Mark Wahlberg 122044.000000 votes 7713515
nm0004266 Anne Hathaway 471724.000000 votes 7363835
nm0350453 Jake Gyllenhaal 562559.000000 votes 7273000
nm0000129 Tom Cruise 584839.000000 votes 7250720
nm0005351 Ryan Reynolds 591712.000000 votes 7243847
nm0177896 Bradley Cooper 601901.000000 votes 7233658
nm0000204 Natalie Portman 713529.000000 votes 7122030
nm0000136 Johnny Depp 764243.000000 votes 8599802
nm0000158 Tom Hanks 775918.000000 votes 7059641

Euclidean distance AFTER adding 'votes' with Mi